In [1]:
import copy
from sklearn.model_selection import KFold
from featureExtraction import featureExtraction
from modelGenerator import modelGenerator
from keras import backend as K 

N_splits = 5
DataID = "raw_data_10000_samples_fm_20000_tests_Prueba_21_Prueba_24_Prueba_27"
modelsID = ["lstm","esn","tcn","seq2point","rocket"]
data = featureExtraction(DataID,statorFreqs=[37],testsID=[21,24],featsDomain="time",timesteps=1100)  # raw_data_10000_samples_fm_20000_tests_Prueba_21_Prueba_24_Prueba_27

2023-07-24 14:09:41.808022: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/diego/.conda/envs/hiwind/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (None)/charset_normalizer (2.1.1) doesn't match a supported version!
  warnings.warn(


In [2]:
METRICS = []

MODEL_LABELS = {"lstm": "LSTM",
                "rocket": "rocket",
                "seq2point":"CNN",
                "tcn": "TCN",
                "esn":"ESN"}

for modelID in modelsID:
    X =copy.deepcopy(data.X)
    y =copy.deepcopy(data.y)

    skf = KFold(n_splits=N_splits,shuffle=True)
    for k, (train_index, test_index) in enumerate(skf.split(X, y)):
            data.X_train,data.y_train = X[train_index],y[train_index]
            data.X_test, data.y_test  = X[test_index],y[test_index]
            model = modelGenerator(modelID=modelID, data=data,params={})
            model.train()
            METRICS.append([MODEL_LABELS[modelID],k,"MAE",model.test_MAE])
            METRICS.append([MODEL_LABELS[modelID],k,"MSE",model.test_MSE])
            del model
            K.clear_session()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 1100, 4)]         0         
                                                                 
 conv1d (Conv1D)             (None, 1086, 64)          3904      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 543, 64)          0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 543, 128)          98816     
                                                                 
 flatten (Flatten)           (None, 69504)             0         
                                                                 
 dense (Dense)               (None, 1)                 69505     


2023-07-24 14:09:53.345452: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-24 14:09:53.590545: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22309 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


                                                                 
Total params: 172,225
Trainable params: 172,225
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300


2023-07-24 14:09:54.705416: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-07-24 14:09:55.485712: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


28/28 [==============================] - 3s 33ms/step - loss: 8977.6016 - mean_squared_error: 8977.6016 - mean_absolute_error: 79.4026 - val_loss: 636.0714 - val_mean_squared_error: 636.0714 - val_mean_absolute_error: 20.9251
Epoch 2/300
28/28 [==============================] - 1s 26ms/step - loss: 814.2891 - mean_squared_error: 814.2891 - mean_absolute_error: 23.3543 - val_loss: 637.8030 - val_mean_squared_error: 637.8030 - val_mean_absolute_error: 21.4108
Epoch 3/300
28/28 [==============================] - 1s 22ms/step - loss: 791.1205 - mean_squared_error: 791.1205 - mean_absolute_error: 22.2481 - val_loss: 636.4213 - val_mean_squared_error: 636.4213 - val_mean_absolute_error: 21.4968
Epoch 4/300
28/28 [==============================] - 1s 28ms/step - loss: 788.5701 - mean_squared_error: 788.5701 - mean_absolute_error: 22.2088 - val_loss: 661.9260 - val_mean_squared_error: 661.9260 - val_mean_absolute_error: 22.6327
Epoch 5/300
28/28 [==============================] - 1s 22ms/step 

In [1]:
import pandas as pd
import numpy as np
df_metrics = pd.DataFrame(METRICS,columns = ['Model','k','METRICS','VALUES'])
table_metrics = df_metrics.pivot_table(index='Model',columns='METRICS',values="VALUES",aggfunc=[np.mean, np.std])
table_metrics = table_metrics.swaplevel(axis=1)
table_metrics.sort_index(axis=1,inplace=True)
table_metrics.columns.names = (None,None)
table_metrics = table_metrics.reindex([MODEL_LABELS[model] for model in modelsID])
print(table_metrics.to_latex(float_format="%1.3f"))

NameError: name 'METRICS' is not defined